基于SentencePiece扩充LLaMa的词表，但是扩充了词表后的下一步该怎么操作呢？如何将新增的token在模型的embedding层和lm_head层初始化呢？

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model_name = "../sentencepiece/llama2-7b-hf" # llama2模型的位置
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

new_tokenizer = AutoTokenizer.from_pretrained("../sentencepiece/merged_tokenizer_hf_test") # 新训练的分词器的位置

我们加载完模型和分词器以及新增的分词器后，看一下模型的结构：
```python
model
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)
```

原来LLaMa的词表是32000，所以Embedding层的大小为(32000, 4096)，即词表里的每一个token对应一个1*4096的Embedding向量，上篇文章扩充后的词表大小为40114，所以这里有两种方式：
- 随机扩充
- 均值扩充（用的比较多）

#### 随机填充

将扩充的token对应的向量随机初始化，实现方式如下：

In [ ]:
# 获取原先的embedding
embeddings = model.get_input_embeddings()
print(embeddings)
# Embedding(32000, 4096, padding_idx=0)

print(embeddings(torch.LongTensor([31999])))

In [ ]:
model.resize_token_embeddings(41013)
new_embeddings = model.get_input_embeddings()
print(new_embeddings)
# Embedding(41013, 4096)

print(new_embeddings(torch.LongTensor([31999])))

从上述可以看到，Embedding层从32000扩展为了40114，而且前32000个token的Embedding是没有发生变化的，只有新增的token是随机初始化的。

#### 均值扩充

新增token的Embedding用原来token的Embedding的均值来表示，比如比如“你好”在原来的词表里为“你”：[-0.0396, -0.0217, -0.0092, ..., -0.0032, -0.0103, 0.0068]；“好”：[-0.0104, -0.0145, -0.0142, ..., -0.0048, 0.0042, -0.0014]，则新增的“你好”则为其均值：[-0.0250, -0.0181, -0.0117, ..., -0.0040, -0.0030, 0.0027]，以此方式扩充：

In [ ]:
# 新增的token和在原来token相对应的字典
token_mapping = {}
for i in range(32000, len(new_tokenizer)):
    # 使用 tokenizer 的 convert_ids_to_tokens 方法将索引转换为对应的 token
    token = new_tokenizer.convert_ids_to_tokens(i)
    # 原来的token
    input_ids = tokenizer(token, return_tensors="pt").input_ids[0]
    if input_ids[1] == 29871:
        new_input_ids = input_ids[2:]
    else:
        new_input_ids = input_ids[1:]        
    token_mapping[i] = new_input_ids

# 原始输入embedding
embeddings = model.get_input_embeddings()
# 新完全初始化的embedding
new_vocab_size = len(new_tokenizer)
embedding_dim = 4096
new_embedding = torch.nn.Embedding(new_vocab_size, embedding_dim)

# 将现有Embedding层的权重赋值给新的Embedding层的前32000行
num_to_copy = min(new_vocab_size, len(embeddings.weight))
new_embedding.weight.data[:num_to_copy, :] = embeddings.weight.data[:num_to_copy, :]

# 开始新增
for new_token, original_tokens in token_mapping.items():
    original_embeddings = embeddings(original_tokens)
    mean_embedding = torch.mean(original_embeddings, dim=0)
    new_embedding.weight.data[new_token] = mean_embedding

# 更换嵌入层
model.set_input_embeddings(new_embedding)

同理模型的最后一层lm_head: Linear(in_features=4096, out_features=32000, bias=False)，参数也是一个32000*4096的矩阵，方法和上述是一致的，我们来看看均值扩充的方式：

In [ ]:
output_size = 32000
new_output_size = 41013
lm_head = model.lm_head
# 新的lm_head
new_lm_head = torch.nn.Linear(in_features=4096, out_features=new_output_size, bias=False)
# 前32000个向量不变
new_lm_head.weight.data[:output_size, :] = lm_head.weight.data[:output_size, :]

# 新增
for new_token, original_tokens in token_mapping.items():
    original = 0
    for i in original_tokens:
        original += lm_head.weight.data[i]
    mean_para = original / len(original_tokens)
    new_lm_head.weight.data[new_token] = mean_para

# 替换模型原来的lm_head
model.lm_head = new_lm_head
model.config.vocab_size = 41013

# 最后完成了embedding和lm_head替换后，保存模型
model.save_pretrained("llama-2-7b-extent", max_shard_size="8GB")
# 保存为bin格式
#model.save_pretrained("llama-2-7b-extent", max_shard_size="8GB", safe_serialization=false)

上述就是扩充词表后，需要对模型的embedding和lm_head做的操作，目前业界一般用的都是均值的方式。我们可以算一下新增了多少个参数：（41013-32000）* 4096 * 2 = 73,834,496，7千多万个参数，还只是扩充9千个词，如果扩充的词表数量达到5万左右，那新增参数就是1亿多，这个参数数量还是不少的。

初始化了之后，还得预训练，也得更新这个embedding矩阵，性价比比较高的是把训练词表的语料训练一遍，然后做SFT